In [5]:
import requests
import pandas as pd
import json
import xmltodict

In [8]:
url = "https://api.entur.io/realtime/v1/rest/vm?datasetId=SKY"

response = requests.get(url)

In [34]:
data = xmltodict.parse(response.text)["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"]["VehicleActivity"]
#print(json.dumps(data, indent=4))

monitoredVehicleJourneyData = [x["MonitoredVehicleJourney"] for x in data]
df = pd.DataFrame(monitoredVehicleJourneyData)
#df.to_csv("bus_data.csv", index=False)

print(df["VehicleLocation"][318])

{'Longitude': '5.16912999562919', 'Latitude': '60.4455000255257'}
